

```
# Ejecución del proceso de clonacion e inicialización del repositorio
```



In [72]:
!git clone https://github.com/maragr-sudo/Python-Web-Scraping.git

fatal: destination path 'Python-Web-Scraping' already exists and is not an empty directory.


In [73]:
!git init

Reinitialized existing Git repository in /content/.git/


In [74]:
!git config user.name maragr-sudo
!git config user.email marthi1901@gmail.com

# Web Scraping with Beautiful Soup

* * *

### Icons used in this notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
🥊 **Challenge**: Interactive exercise. We'll work through these in the workshop!<br>
⚠️ **Warning**: Heads-up about tricky stuff or common mistakes.<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
🎬 **Demo**: Showing off something more advanced – so you know what Python can be used for!<br>

### Learning Objectives
1. [Reflection: To Scape Or Not To Scrape](#when)
2. [Extracting and Parsing HTML](#extract)
3. [Scraping the Illinois General Assembly](#scrape)

<a id='when'></a>

# To Scrape Or Not To Scrape

When we'd like to access data from the web, we first have to make sure if the website we are interested in offers a Web API. Platforms like Twitter, Reddit, and the New York Times offer APIs. **Check out D-Lab's [Python Web APIs](https://github.com/dlab-berkeley/Python-Web-APIs) workshop if you want to learn how to use APIs.**

However, there are often cases when a Web API does not exist. In these cases, we may have to resort to web scraping, where we extract the underlying HTML from a web page, and directly obtain the information we want. There are several packages in Python we can use to accomplish these tasks. We'll focus two packages: Requests and Beautiful Soup.

Our case study will be scraping information on the [state senators of Illinois](http://www.ilga.gov/senate), as well as the [list of bills](http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True) each senator has sponsored. Before we get started, peruse these websites to take a look at their structure.

**` Objetivo del notebook`**

*El objetivo principal de este notebook es enseñar los fundamentos del web scraping utilizando las bibliotecas de Python requests y BeautifulSoup*




🧰 **Sección 1: Instalacion de librerias requeridas**


## Installation

We will use two main packages: [Requests](http://docs.python-requests.org/en/latest/user/quickstart/) and [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Go ahead and install these packages, if you haven't already:

-- Se instala **requests** para hacer peticiones HTTP y obtener el contenido de una página web,  **BeautifulSoup** nos permite analizar y extraer información del HTML, y se instala tambien **lxml** es un analizador de alto rendimiento que trabaja con Beautiful Soup para procesar el HTML de manera más eficiente.

In [75]:
%pip install requests

In [76]:
%pip install beautifulsoup4

We'll also install the `lxml` package, which helps support some of the parsing that Beautiful Soup performs:

In [77]:
%pip install lxml

In [78]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

---*Estas librerías permiten hacer peticiones web, analizar HTML, manejar datos y visualizarlos*.

<a id='extract'></a>

# Extracting and Parsing HTML

In order to succesfully scrape and analyse HTML, we'll be going through the following 4 steps:
1. Make a GET request
2. Parse the page with Beautiful Soup
3. Search for HTML elements
4. Get attributes and text of these elements

## Step 1: Make a GET Request to Obtain a Page's HTML

We can use the Requests library to:

1. Make a GET request to the page, and
2. Read in the webpage's HTML code.

The process of making a request and obtaining a result resembles that of the Web API workflow. Now, however, we're making a request directly to the website, and we're going to have to parse the HTML ourselves. This is in contrast to being provided data organized into a more straightforward `JSON` or `XML` output.

## **🧰 Paso 1: Obtener el HTML de una Página**
El primer paso del proceso de web scraping es descargar el código fuente de la página http://www.ilga.gov/senate/default.asp , el código utiliza requests.get() para hacer una solicitud a la URL . El resultado de la solicitud se guarda en la variable req, y el contenido de texto (HTML) se extrae usando req.text. El código luego imprime los primeros 1000 caracteres del HTML para mostrar una vista previa del contenido que se ha descargado.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# View some output
print(src[:1000])

## **🧰 Paso 2: Método BeautifulSoup: Construcción y análisis de la página principal**

## Step 2: Parse the Page with Beautiful Soup

Now, we use the `BeautifulSoup` function to parse the reponse into an HTML tree. This returns an object (called a **soup object**) which contains all of the HTML in the original document.

If you run into an error about a parser library, make sure you've installed the `lxml` package to provide Beautiful Soup with the necessary parsing tools.

In [ ]:
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')
# Take a look
print(soup.prettify()[:1000])


---El código crea un objeto BeautifulSoup al que se le pasa el HTML descargado (src) y el analizador ('lxml').
El método soup.prettify() se utiliza para formatear el HTML de manera legible, mostrando el árbol de elementos.---


The output looks pretty similar to the above, but now it's organized in a `soup` object which allows us to more easily traverse the page.

### **🧰 Paso 3: Buscar Elementos HTML**

## Step 3: Search for HTML Elements

Beautiful Soup has a number of functions to find useful components on a page. Beautiful Soup lets you find elements by their:

1. HTML tags
2. HTML Attributes
3. CSS Selectors

Let's search first for **HTML tags**.

The function `find_all` searches the `soup` tree to find all the elements with an a particular HTML tag, and returns all of those elements.

What does the example below do?

Esta es la parte clave para encontrar la información deseada. El notebook muestra tres formas principales de buscar elementos:

Por etiqueta HTML: El método find_all("a") busca todos los elementos de

*   Por etiqueta HTML: El método find_all("a") busca todos los elementos de anclaje.

*   Por clase CSS: Se puede combinar la búsqueda por etiqueta con la clase CSS. Por ejemplo, soup.select("a.sidemenu") busca todas las etiquetasque tienen la clase sidemenu.

*   Por selector CSS: El método select() permite usar selectores CSS más
complejos para obtener elementos de forma muy específica, como soup.select('tr tr tr') para encontrar filas de tabla anidadas.

In [ ]:
# Find all elements with a certain tag
a_tags = soup.find_all("a")
print(a_tags[:10])

Because `find_all()` is the most popular method in the Beautiful Soup search API, you can use a shortcut for it. If you treat the BeautifulSoup object as though it were a function, then it’s the same as calling `find_all()` on that object.

These two lines of code are equivalent:

In [ ]:
a_tags = soup.find_all("a")
a_tags_alt = soup("a")
print(a_tags[0])
print(a_tags_alt[0])

How many links did we obtain?

In [ ]:
print(len(a_tags))

That's a lot! Many elements on a page will have the same HTML tag. For instance, if you search for everything with the `a` tag, you're likely to get more hits, many of which you might not want. Remember, the `a` tag defines a hyperlink, so you'll usually find many on any given page.

What if we wanted to search for HTML tags with certain attributes, such as particular CSS classes?

We can do this by adding an additional argument to the `find_all`. In the example below, we are finding all the `a` tags, and then filtering those with `class_="sidemenu"`.

In [ ]:
# Get only the 'a' tags in 'sidemenu' class
side_menus = soup("a", class_="sidemenu")
side_menus[:5]

A more efficient way to search for elements on a website is via a **CSS selector**. For this we have to use a different method called `select()`. Just pass a string into the `.select()` to get all elements with that string as a valid CSS selector.

In the example above, we can use `"a.sidemenu"` as a CSS selector, which returns all `a` tags with class `sidemenu`.

In [ ]:
# Get elements with "a.sidemenu" CSS Selector.
selected = soup.select("a.sidemenu")
selected[:5]

## 🥊 Challenge: Find All

Use BeautifulSoup to find all the `a` elements with class `mainmenu`.

In [ ]:
# MY CODE HERE
main_menus = soup.find_all('a', class_='mainmenu')
main_menus[:4]

### **🧰 Paso 4: Extraer Atributos y Texto**


## Step 4: Get Attributes and Text of Elements

Once we identify elements, we want the access information in that element. Usually, this means two things:

1. Text
2. Attributes

Getting the text inside an element is easy. All we have to do is use the `text` member of a `tag` object:

-- Texto: Se accede al texto dentro de un elemento usando la propiedad .text.

-- Atributos: Los atributos de un elemento (como el href en un enlace) se pueden obtener tratando el objeto de la etiqueta como un diccionario, por ejemplo, enlace['href']

In [ ]:
# Get all sidemenu links as a list
side_menu_links = soup.select("a.sidemenu")

# Examine the first link
first_link = side_menu_links[0]
print(first_link)

# What class is this variable?
print('Class: ', type(first_link))

It's a Beautiful Soup tag! This means it has a `text` member:

In [ ]:
print(first_link.text)

Sometimes we want the value of certain attributes. This is particularly relevant for `a` tags, or links, where the `href` attribute tells us where the link goes.

💡 **Tip**: You can access a tag’s attributes by treating the tag like a dictionary:

In [ ]:
print(first_link['href'])

## 🥊 Challenge: Extract specific attributes

Extract all `href` attributes for each `mainmenu` URL.

In [ ]:
# MY CODE HERE
mainmenu_links = [a['href'] for a in soup.select('a.mainmenu') if a.has_attr('href')]
print(mainmenu_links[:10])

<a id='scrape'></a>

# Scraping the Illinois General Assembly

Believe it or not, those are really the fundamental tools you need to scrape a website. Once you spend more time familiarizing yourself with HTML and CSS, then it's simply a matter of understanding the structure of a particular website and intelligently applying the tools of Beautiful Soup and Python.

Let's apply these skills to scrape the [Illinois 98th General Assembly](http://www.ilga.gov/senate/default.asp?GA=98).

Specifically, our goal is to scrape information on each senator, including their name, district, and party.

## Scrape and Soup the Webpage

Let's scrape and parse the webpage, using the tools we learned in the previous section.

***#Descarga la página objetiva***

> Añadir blockquote



In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")

## Search for the Table Elements

Our goal is to obtain the elements in the table on the webpage. Remember: rows are identified by the `tr` tag. Let's use `find_all` to obtain these elements.

In [ ]:
# Get all table row elements
rows = soup.find_all("tr")
len(rows)

⚠️ **Warning**: Keep in mind: `find_all` gets *all* the elements with the `tr` tag. We only want some of them. If we use the 'Inspect' function in Google Chrome and look carefully, then we can use some CSS selectors to get just the rows we're interested in. Specifically, we want the inner rows of the table:

In [ ]:
# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

for row in rows[:5]:
    print(row, '\n')

***# Se usa selector ¨tr tr tr¨para ir a filas anidadas internas (donde está la data útil) y muestra ejemplos.***

It looks like we want everything after the first two rows. Let's work with a single row to start, and build our loop from there.

In [ ]:
example_row = rows[2]
print(example_row.prettify())

Let's break this row down into its component cells/columns using the `select` method with CSS selectors. Looking closely at the HTML, there are a couple of ways we could do this.

* We could identify the cells by their tag `td`.
* We could use the the class name `.detail`.
* We could combine both and use the selector `td.detail`.

***# Muestra tres formas equivalentes de seleccionar celdas***

In [ ]:
for cell in example_row.select('td'):
    print(cell)
print()

for cell in example_row.select('.detail'):
    print(cell)
print()

for cell in example_row.select('td.detail'):
    print(cell)
print()

We can confirm that these are all the same.

In [ ]:
assert example_row.select('td') == example_row.select('.detail') == example_row.select('td.detail')

Let's use the selector `td.detail` to be as specific as possible.

In [ ]:
# Select only those 'td' tags with class 'detail'
detail_cells = example_row.select('td.detail')
detail_cells

Most of the time, we're interested in the actual **text** of a website, not its tags. Recall that to get the text of an HTML element, we use the `text` member:

In [ ]:
# Keep only the text in each of those cells
row_data = [cell.text for cell in detail_cells]

print(row_data)

Looks good! Now we just use our basic Python knowledge to get the elements of this list that we want. Remember, we want the senator's name, their district, and their party.

***#Se detalla  qué campos los interesa imprimir : nombre (0), distrito (3) y partido (4).***

In [ ]:
print(row_data[0]) # Name
print(row_data[3]) # District
print(row_data[4]) # Party

## Getting Rid of Junk Rows

We saw at the beginning that not all of the rows we got actually correspond to a senator. We'll need to do some cleaning before we can proceed forward. Take a look at some examples:

In [ ]:
print('Row 0:\n', rows[0], '\n')
print('Row 1:\n', rows[1], '\n')
print('Last Row:\n', rows[-1])

When we write our for loop, we only want it to apply to the relevant rows. So we'll need to filter out the irrelevant rows. The way to do this is to compare some of these to the rows we do want, see how they differ, and then formulate that in a conditional.

As you can imagine, there a lot of possible ways to do this, and it'll depend on the website. We'll show some here to give you an idea of how to do this.

***#Nos propone formular una condición para quedarnos solo con filas válidas***

In [ ]:
# Bad rows
print(len(rows[0]))
print(len(rows[1]))

# Good rows
print(len(rows[2]))
print(len(rows[3]))

Perhaps good rows have a length of 5. Let's check:

In [ ]:
good_rows = [row for row in rows if len(row) == 5]

# Let's check some rows
print(good_rows[0], '\n')
print(good_rows[-2], '\n')
print(good_rows[-1])

We found a footer row in our list that we'd like to avoid. Let's try something else:

In [ ]:
rows[2].select('td.detail')

In [ ]:
# Bad row
print(rows[-1].select('td.detail'), '\n')

# Good row
print(rows[5].select('td.detail'), '\n')

# How about this?
good_rows = [row for row in rows if row.select('td.detail')]

print("Checking rows...\n")
print(good_rows[0], '\n')
print(good_rows[-1])

Looks like we found something that worked!

## Loop it All Together

Now that we've seen how to get the data we want from one row, as well as filter out the rows we don't want, let's put it all together into a loop.

In [ ]:
# Define storage list
members = []

# Get rid of junk rows
valid_rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in valid_rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    # Store in a tuple
    senator = (name, district, party)
    # Append to list
    members.append(senator)

In [ ]:
# Should be 61
len(members)

Let's take a look at what we have in `members`.

In [ ]:
print(members[:5])

## 🥊  Challenge: Get `href` elements pointing to members' bills

The code above retrieves information on:  

- the senator's name,
- their district number,
- and their party.

We now want to retrieve the URL for each senator's list of bills. Each URL will follow a specific format.

The format for the list of bills for a given senator is:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

to get something like:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

in which `MEMBER_ID=1911`.

You should be able to see that, unfortunately, `MEMBER_ID` is not currently something pulled out in our scraping code.

Your initial task is to modify the code above so that we also **retrieve the full URL which points to the corresponding page of primary-sponsored bills**, for each member, and return it along with their name, district, and party.

Tips:

* To do this, you will want to get the appropriate anchor element (`<a>`) in each legislator's row of the table. You can again use the `.select()` method on the `row` object in the loop to do this — similar to the command that finds all of the `td.detail` cells in the row. Remember that we only want the link to the legislator's bills, not the committees or the legislator's profile page.
* The anchor elements' HTML will look like `<a href="/senate/Senator.asp/...">Bills</a>`. The string in the `href` attribute contains the **relative** link we are after. You can access an attribute of a BeatifulSoup `Tag` object the same way you access a Python dictionary: `anchor['attributeName']`. See the <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> for more details.
* There are a _lot_ of different ways to use BeautifulSoup to get things done. whatever you need to do to pull the `href` out is fine.

The code has been partially filled out for you. Fill it in where it says `#YOUR CODE HERE`. Save the path into an object called `full_path`.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Create empty list to store our data
members = []

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
# Get rid of junk rows
rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]

    # YOUR CODE HERE
    full_path = ''

    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

In [ ]:
# Uncomment to test
# members[:5]

## 🥊  Challenge: Modularize Your Code

Turn the code above into a function that accepts a URL, scrapes the URL for its senators, and returns a list of tuples containing information about each senator.

In [ ]:
# YOUR CODE HERE
def get_members(url):
    return [___]


In [ ]:
# Test your code
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = get_members(url)
len(senate_members)

## 🥊 Take-home Challenge: Writing a Scraper Function

We want to scrape the webpages corresponding to bills sponsored by each bills.

Write a function called `get_bills(url)` to parse a given bills URL. This will involve:

  - requesting the URL using the <a href="http://docs.python-requests.org/en/latest/">`requests`</a> library
  - using the features of the `BeautifulSoup` library to find all of the `<td>` elements with the class `billlist`
  - return a _list_ of tuples, each with:
      - description (2nd column)
      - chamber (S or H) (3rd column)
      - the last action (4th column)
      - the last action date (5th column)
      
This function has been partially completed. Fill in the rest.

In [ ]:
def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src)
    rows = soup.select('tr')
    bills = []
    for row in rows:
        # YOUR CODE HERE
        bill_id =
        description =
        chamber =
        last_action =
        last_action_date =
        bill = (bill_id, description, chamber, last_action, last_action_date)
        bills.append(bill)
    return bills

In [ ]:
# Uncomment to test your code
# test_url = senate_members[0][3]
# get_bills(test_url)[0:5]

### Scrape All Bills

Finally, create a dictionary `bills_dict` which maps a district number (the key) onto a list of bills (the value) coming from that district. You can do this by looping over all of the senate members in `members_dict` and calling `get_bills()` for each of their associated bill URLs.

**NOTE:** please call the function `time.sleep(1)` for each iteration of the loop, so that we don't destroy the state's web site.

In [ ]:
# MY CODE HERE
import time
bills_dict = {}
for name, district, party, bills_url in senate_members:
    if not bills_url:
        continue
    bills = get_bills(bills_url)
    bills_dict[district] = bills
    time.sleep(1) # respetar el servidor


# Ejemplo
print(list(bills_dict.keys())[:5])
print(bills_dict[52][:2])

In [ ]:
# Uncomment to test your code
# bills_dict[52]

En conclusión esta notebook nos enseña a descargar una página web (requests),  analizar el HTML (BeautifulSoup) y a extraer información (títulos, subtítulos, enlaces, tablas).